We want to look at the evolution of probabilities after quantum gates have acted upon them. The corresponding formula for them is:

In [1]:
from IPython.display import Image
#Image("Probabilistic_operators.png")

Now let us find an expression for the overlap matrix $O_{\textbf{a'',a'}}$. We shall begin in the tetrahedral POVM (this serves as a specific input), but the following code, including all Operators etc., will be such that it is modular and can be adapted for other POVM basis choices.

In [2]:
## import necessary packages ##

import numpy.linalg as lin
import numpy as np


In [3]:
## Define the fundamental 2x2 matrices ##

sig_x = np.array([ [0, 1], 
                   [1, 0] ])

sig_y = np.array([ [0, -1j],
                   [1j, 0] ])

sig_z = np.array([ [1,0],
                   [0,-1] ])

Id_2x2 = np.array([ [1, 0], 
                       [0, 1] ])

pauli_mats = np.array([sig_x, sig_y, sig_z])

print(np.shape(pauli_mats))

(3, 2, 2)


In [4]:
## define an arbitrary POVM measurement (bloch representation)

def construct_POVM(a, b, n):
    # a, b are scalars corresponding to normalization, n is projection vector (normalized)
    
    return( a * Id_2x2 + b * np.einsum("i,ijk->jk", n, pauli_mats) )

In [5]:
## Define Rotation Matrix for arbitrary axis
 
def rotation_matrix(axis, theta):
    """
    Return the rotation matrix associated with counterclockwise rotation about
    the given axis by theta radians.
    """
    axis = np.asarray(axis)
    axis = axis / np.sqrt(np.dot(axis, axis)) #normalize
    a = np.cos(theta / 2.0)
    b, c, d = -axis * np.sin(theta / 2.0)
    aa, bb, cc, dd = a * a, b * b, c * c, d * d
    bc, ad, ac, ab, bd, cd = b * c, a * d, a * c, a * b, b * d, c * d
    return np.array([[aa + bb - cc - dd, 2 * (bc + ad), 2 * (bd - ac)],
                     [2 * (bc - ad), aa + cc - bb - dd, 2 * (cd + ab)],
                     [2 * (bd + ac), 2 * (cd - ab), aa + dd - bb - cc]])

In [6]:
## Two POVM symmetry angles (Theta: x, Phi:z)

def rotation_matrix_ThetPhi(theta, phi):
    return rotation_matrix([1, 0, 0], theta) @ rotation_matrix([0, 0, 1], phi)


In [7]:
## specify tetrahedral POVM

# apply rotations here (theta, phi)
n_1 = np.array([0, 0, 1]).T
n_2 = np.array([2*np.sqrt(2)/3, 0, -1/3]).T
n_3 = np.array([-np.sqrt(2)/3, np.sqrt(2/3), -1/3]).T
n_4 = np.array([-np.sqrt(2)/3, -np.sqrt(2/3), -1/3]).T

n_tetra = np.array([n_1, n_2, n_3, n_4])

n_tetra_rot = [rotation_matrix_ThetPhi(0.03, 0.02) @ n for n in n_tetra]

for i in range(3):
    # check if all vectors were rotated properly
    print(np.abs(np.dot(n_tetra_rot[i], n_tetra_rot[i + 1])) - np.abs(np.dot(n_tetra[i], n_tetra[i + 1])))
    
    
## Construct two POVMs

# Original POVM
POVM_tetra = [construct_POVM(1/4, 1/4, n_vec) for n_vec in n_tetra]

# Rotatet POVM for strictly non-zero entries
POVM_tetra_rot = [construct_POVM(1/4, 1/4, n_vec) for n_vec in n_tetra_rot]


-5.551115123125783e-17
-5.551115123125783e-17
1.1102230246251565e-16


The deviation of the magnitude of the overlap between two POVM elements is = 0. (Visually) check if they still span a tetrahedron by plotting the vectors:

In [8]:
%%capture
%matplotlib notebook

import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

import numpy as np
from numpy import *
from matplotlib import pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from matplotlib.patches import FancyArrowPatch
from mpl_toolkits.mplot3d import proj3d

class Arrow3D(FancyArrowPatch):
    def __init__(self, xs, ys, zs, *args, **kwargs):
        FancyArrowPatch.__init__(self, (0,0), (0,0), *args, **kwargs)
        self._verts3d = xs, ys, zs

    def draw(self, renderer):
        xs3d, ys3d, zs3d = self._verts3d
        xs, ys, zs = proj3d.proj_transform(xs3d, ys3d, zs3d, renderer.M)
        self.set_positions((xs[0],ys[0]),(xs[1],ys[1]))
        FancyArrowPatch.draw(self, renderer)
        
        
fig = plt.figure(figsize=(10, 10))
ax = fig.add_subplot(111, projection='3d')

for i in range(4):
    a = Arrow3D([0, n_tetra[i][0]], [0, n_tetra[i][1]], 
                [0, n_tetra[i][2]], mutation_scale=20, 
                lw=3, arrowstyle="-|>", color="r")
    b = Arrow3D([0, n_tetra_rot[i][0]], [0, n_tetra_rot[i][1]], 
                [0, n_tetra_rot[i][2]], mutation_scale=20, 
                lw=3, arrowstyle="-|>", color="b")
    ax.add_artist(a)
    ax.add_artist(b)

ax.set_xlabel('x_values')
ax.set_ylabel('y_values')
ax.set_zlabel('z_values')

ax.set_xlim(-1, 1)
ax.set_ylim(-1, 1)
ax.set_zlim(-1, 1)

plt.title('POVM Tetrahedron before and after Rotation')

plt.draw()
plt.show()


In [9]:
## build many-body POVM measurements

def many_body_POVM(outcomes, POVM):
    # takes a vector of POVM outcomes and constructs the corresponding 
    # many-body measurement
    
    mb_operator = POVM[outcomes[0] - 1]
    
    for outcome in outcomes[1: ]:
        mb_operator = np.kron(mb_operator, POVM[outcome - 1])
        
    return(mb_operator)

The overlap matrix between POVM measurement operators for $\textbf{a'}$ and $\textbf{a''}$ is given by:

In [10]:
Image("overlap.png")

FileNotFoundError: No such file or directory: 'overlap.png'

FileNotFoundError: No such file or directory: 'overlap.png'

<IPython.core.display.Image object>

In [11]:
## construct the overlap matrix

In [35]:
## first construct all possible POVM measurements

n_qubits = 2

n_measurements = 4**n_qubits

from itertools import product

measurements_test = list(product([1, 2, 3, 4], repeat=n_qubits))
%store measurements_test

Stored 'measurements_test' (list)


In [13]:
def outcome_to_ID(outcome, measurements):
    # assign each measurement outcome a unique index based
    # on the outcome generating function
    return measurements.index(outcome)

def ID_to_outcome(ID, measurements):
    # get the measurement outcome based on its ID value
    return measurements[ID]

In [14]:
def overlap(n_measurements, measurements, POVM_choice):
    # POVM_choice: string specifying which POVM to use
    if POVM_choice == "Tetra":
        POVM = POVM_tetra
        
    elif POVM_choice == "Tetra_rot":
        POVM = POVM_tetra_rot
        
    overlap_matrix = np.zeros((n_measurements, n_measurements), dtype=np.complex_)

    for i in range(n_measurements):

        for j in range(n_measurements):

            M_i = many_body_POVM(ID_to_outcome(i, measurements), POVM)
            M_j = many_body_POVM(ID_to_outcome(j, measurements), POVM)

            overlap_matrix[i, j] = np.trace(M_i @ M_j)
            
    return(overlap_matrix)

In [15]:
overlap_tetra = overlap(n_measurements, measurements, "Tetra")
overlap_tetra_rot = overlap(n_measurements, measurements, "Tetra_rot")

Now we simply need to find the matrix inverse:

In [16]:
overlap_tetra_inv = np.linalg.inv(overlap_tetra)
overlap_tetra_rot_inv = np.linalg.inv(overlap_tetra_rot)

In [17]:
print(overlap_tetra_rot)

[[0.25      +0.j 0.08333333+0.j 0.08333333+0.j 0.08333333+0.j]
 [0.08333333+0.j 0.25      +0.j 0.08333333+0.j 0.08333333+0.j]
 [0.08333333+0.j 0.08333333+0.j 0.25      +0.j 0.08333333+0.j]
 [0.08333333+0.j 0.08333333+0.j 0.08333333+0.j 0.25      +0.j]]


This agrees with the result obtained in the paper of Carrasquila (https://arxiv.org/pdf/1810.10584.pdf).

In [18]:
overlap_tetra_inv @ overlap_tetra

array([[ 1.00000000e+00+0.j, -1.43403807e-16+0.j, -3.23815049e-17+0.j,
        -5.55111512e-17+0.j],
       [ 4.16333634e-17+0.j,  1.00000000e+00+0.j,  1.38777878e-17+0.j,
         2.77555756e-17+0.j],
       [-1.85037171e-17+0.j,  3.70074342e-17+0.j,  1.00000000e+00+0.j,
        -5.55111512e-17+0.j],
       [-2.77555756e-17+0.j,  5.55111512e-17+0.j,  1.38777878e-17+0.j,
         1.00000000e+00+0.j]])

Indeed the overlap matrix could be inverted.

Now we still need to implement the quantum gates:

In [19]:
# definition of 1-qubit quantum gates

ID = np.array([[1, 0],    # Identity Gate
               [0, 1]])

X = sig_x    #Pauli-X

Y = sig_y    #Pauli-Y

Z = sig_z    #Pauli-Z

H = 1/np.sqrt(2) * np.array([[1, 1],    #Hadamard
                             [1, -1]])    

S = np.array([[1, 0],    #Phase
              [0, 1j]])

Pi8 = np.array([[1, 0],    #Pi/8
                [0, np.exp(1j*np.pi/4)]])

We need a method of generalising these 1-qubit states to act on one specific qubit from a setup of multiple ones

In [20]:
import scipy.sparse as sparse

def buildSparseGateSingle(n, i, gate):
    sgate = sparse.csr_matrix(gate)
    return sparse.kron(sparse.kron(sparse.identity(2**i), sgate), sparse.identity(2**(n-i-1)))

def buildGateSingle(n_qubits, i, gate):
    single_qubit_gates = [ID]*i + [gate] + [ID]*(n_qubits-i-1) 
    
    total_gate = single_qubit_gates[0]
    
    for gate in single_qubit_gates[1:] :
        total_gate = np.kron(total_gate, gate)

    return(total_gate)

In [21]:
def dagger(operator):
    return operator.conj().T

Now we finally have all the tools needed for the construction of the operator matrix in the POVM formalism

In [22]:
def prob_operator(n_qubits, POVM_type, n_outcomes, gate, gate_index):
    
    n_mb_outcomes = n_outcomes**n_qubits
    measurements = list(product([1, 2, 3, 4], repeat=n_qubits))

    
    if POVM_type == "Tetra":
        POVM = POVM_tetra
        
    elif POVM_type == "Tetra_rot":
        POVM = POVM_tetra_rot

    overlap_mat = overlap(n_mb_outcomes, measurements, POVM_type)
    overlap_inv = np.linalg.inv(overlap_mat)
        
    mb_gate = buildGateSingle(n_qubits, gate_index, gate)
    
    O_mat = np.zeros((n_mb_outcomes, n_mb_outcomes), dtype=np.complex_)
    
    for i in range(n_mb_outcomes):
        for j in range(n_mb_outcomes):
            for k in range(n_mb_outcomes):
                O_mat[i][j] += np.trace(mb_gate @ many_body_POVM(ID_to_outcome(k, measurements), POVM) 
                                        @ dagger(mb_gate) @ many_body_POVM(ID_to_outcome(i, measurements), POVM)) * overlap_inv[k][j] 
                
    
    return(O_mat)

In [23]:
def round_zero(array):
    eps = 1.0e-10
    array[np.abs(array) < eps] = 0
    
    return(array)

# Test of Update Rules

## Plan

done:
* worked out update rules for simple 2-qubit system
* implemented 1-qubit gates in code
* introduce density matrix formalism to code
* introduce function: swap density matrix <--> POVM formalism
* formulate density matrix for simple product state
* calculate rho & P 
    * before 1-qubit-gate
    * after 1-qubit-gate
* calculate P' using the mapping P' = O @ P --> check if mapping is correct


to-do:
* implement simple ANN (autoregressive) to obtain the P-distribution
* train the network for simple input state
* apply gate to network and check if the parameter update gives correct P'
    * for product state
    * for entangled state (W != 0)

1. Density matrix for pure state

In [24]:
def init_density_mat(states, n_qubits):
    """ Code for product states and entangled states.
    states is list of tuples (probability, state)
    
    If product state:
        state is list of np.arrays corresponding to subsystem states. (!This is prduct state property!)
    Elif entangled state:
        state is already a 2^n_qubits - dimensional vector corresponding to the state vector of the entangled system"""
    
    rho = np.zeros((2**n_qubits, 2**n_qubits), dtype=np.complex_)
    
    for state in states:
        p = state[0]
        state_info = state[1]
         
        if len(state_info) != 1: #product state
            mb_state = state_info[0]

            for vec in state_info[1: ]:
                mb_state = np.kron(mb_state, vec)
                
        elif len(state_info) == 1: #entangled state
            mb_state = state_info[0]
        
            
        rho += p * mb_state @ np.conjugate(mb_state).T
        
    return(rho)

In [25]:
def all_mb_POVM_ops(n_qubits, POVM_type):
    # n_qubits: number of qubits in the system
    # POVM_type (string): name of POVM
    
    measurements = list(product([1, 2, 3, 4], repeat=n_qubits))

    if POVM_type == "Tetra":
        POVM = POVM_tetra
        
    elif POVM_type == "Tetra_rot":
        POVM = POVM_tetra_rot
    
    all_mb_ops = []
    
    for outcome in measurements:
        all_mb_ops.append(many_body_POVM(outcome, POVM))
        
    return(all_mb_ops)
    

In [26]:
def rho_to_POVM_distr(rho, n_qubits, POVM_type, n_outcomes):
    # takes a density matrix and returns the corresponding POVM measurement distribution
    
    # rho: The density matrix of the system of interest
    # n_qubits: The number of qubits in our system
    # POVM_type (string): name of the POVM
    # n_outcomes: the number of operators in the POVM_type array
    
    POVM_ops = all_mb_POVM_ops(n_qubits, POVM_type) #all POVM operators = n_outcomes**n_qubits
    
    P = np.zeros((n_outcomes**n_qubits, 1), dtype=np.complex_)
    
    for i in range(len(P)):
        P[i] = np.trace(rho @ POVM_ops[i])
    
    return(P)
    

In [27]:
def POVM_distr_to_rho(POVM_distr, n_qubits, POVM_type, n_outcomes):
    # takes a POVM probability distribution and maps it to a density matrix
    
    #POVM_distr: an n_outcomes**n_qubits dimensional vector with Probability for each outcome
    #n_qubits: number of qubits in our system
    #POVM_type (string): specifies which POVM we use
    #n_outcomes: nr of different POVM_outcomes (depends on POVM_type)
    
    POVM_ops = all_mb_POVM_ops(n_qubits, POVM_type) #all POVM operators = n_outcomes**n_qubits
    
    measurements = list(product([1, 2, 3, 4], repeat=n_qubits))
    %store measurements
        

    T_inv = np.linalg.inv(overlap(n_outcomes**n_qubits, measurements, POVM_type))
    
    rho_b = T_inv @ POVM_distr
    #rho = np.einsum("ijk,i->jk",POVM_ops, rho_b)
    rho = np.moveaxis(POVM_ops, 0, -1) @ rho_b
    
    return(np.reshape(rho, (n_outcomes, n_outcomes)))

We can conclude that all of this works fine, just as expected!

Now we will formulate a simple 2-qubit (product state) test setting for our quantum circuit.

In [28]:
up = np.array([[1, 0]]).T
down = np.array([[0, 1]]).T

plus = 1/np.sqrt(2) * np.array([[1, 1]]).T
minus = 1/np.sqrt(2) * np.array([[1, -1]]).T

plus_i = 1/np.sqrt(2) * np.array([[1, 1.0j]]).T
minus_i = 1/np.sqrt(2) * np.array([[1, -1.0j]]).T

## Density matrix and outcome distribution before gate

# generate several for update biases testing with more complicated product states

# (plus,minus)
rho_init_pm = init_density_mat([(1, [plus, minus])], 2)                            
P_init_pm = rho_to_POVM_distr(rho_init_pm, 2, "Tetra_rot", 4)
%store P_init_pm

# (plus_i,minus_i)
rho_init_pimi = init_density_mat([(1, [plus_i, minus_i])], 2)                            
P_init_pimi = rho_to_POVM_distr(rho_init_pimi, 2, "Tetra_rot", 4)
%store P_init_pimi

# (up, minus_i)
rho_init_umi = init_density_mat([(1, [up, minus_i])], 2)                            
P_init_umi = rho_to_POVM_distr(rho_init_umi, 2, "Tetra_rot", 4)
%store P_init_umi

# (down, plus)
rho_init_dp = init_density_mat([(1, [down, plus])], 2)                            
P_init_dp = rho_to_POVM_distr(rho_init_dp, 2, "Tetra_rot", 4)
%store P_init_dp



## Density matrix and outcome distribution after gate


# as a gate let us choose a simple Pauli-Z gate

gate = Z
mb_gate = buildGateSingle(2, 1, Z) # construct the gate for the many-qubit circuit

#rho_prime = mb_gate @ rho_init @ np.transpose(mb_gate.conjugate())
#P_prime = prob_operator(2, "Tetra_rot", 4, gate, 1) @ P_init

#check if we correctly derived the operator

#rho_prime_check = POVM_distr_to_rho(P_prime, 2, "Tetra_rot", 4)

AG = prob_operator(2, "Tetra_rot", 4, gate, 1)

P_op_X = prob_operator(2, "Tetra_rot", 4, X, 1)
P_op_Y = prob_operator(2, "Tetra_rot", 4, Y, 1)
P_op_Z = prob_operator(2, "Tetra_rot", 4, Z, 1)

P_op_H = prob_operator(2, "Tetra_rot", 4, H, 1)

P_op_phase = prob_operator(2, "Tetra_rot", 4, S, 1)
## save for comparison

#%store P_prime
#%store P_init
%store P_op_X 
%store P_op_Y 
%store P_op_Z 
%store P_op_H 

Stored 'P_init_pm' (ndarray)
Stored 'P_init_pimi' (ndarray)
Stored 'P_init_umi' (ndarray)
Stored 'P_init_dp' (ndarray)
Stored 'P_op_X' (ndarray)
Stored 'P_op_Y' (ndarray)
Stored 'P_op_Z' (ndarray)
Stored 'P_op_H' (ndarray)


In [31]:
np.savetxt("pm_initial_POVM_distro.csv", np.real(P_init_pm), delimiter=",") #save initial POVM distribution

In [36]:
P_prime_pm = P_op_H @ P_init_pm
%store P_prime_pm
np.savetxt("pm_afterH1_POVM_distro.csv", np.real(P_prime_pm), delimiter=",") #save initial POVM distribution

Stored 'P_prime_pm' (ndarray)


In [30]:
def get_positive_negative(array):
    a, b = np.shape(array)[0], np.shape(array)[1]
    
    new_array = np.zeros((a, b))
    
    for i in range(a):
        for j in range(b):
            if array[i, j] > 0 and np.abs(array[i, j]) >= 1.0e-4:
                new_array[i, j] = 1
            elif array[i, j] < 0 and np.abs(array[i, j]) >= 1.0e-4:
                new_array[i, j] = -1
    return(new_array)

In [31]:
get_positive_negative(P_op_Z)

array([[ 1.,  1.,  1., -1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.],
       [ 1., -1.,  1.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.],
       [ 1.,  1., -1.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.],
       [-1.,  1.,  1., -1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  1.,  1.,  1., -1.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  1., -1.,  1.,  1.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  1.,  1., -1.,  1.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.],
       [ 0.,  0.,  0.,  0., -1.,  1.,  1., -1.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  1.,  1., -1.,  0.,
         0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1., -1.,  1.,  1.,  0.,
         0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0

Now we will also formulate product states:

In [32]:
def GHZ(n_qubits):
    state = np.zeros((2**n_qubits,1))
    state[0] = 1
    state[-1] = 1
    print(state)
    print(state.T)
    return(1/np.sqrt(2)*state)

def Bell():
    return(GHZ(2))

GHZ = GHZ(2)

rho_init_GHZ = init_density_mat([(1, [GHZ])], 2)                            

P_init_GHZ =  rho_to_POVM_distr(rho_init_GHZ, 2, "Tetra_rot", 4)
%store P_init_GHZ

P_init_GHZ

[[1.]
 [0.]
 [0.]
 [1.]]
[[1. 0. 0. 1.]]
Stored 'P_init_GHZ' (ndarray)


array([[0.12488753+0.j],
       [0.04177482+0.j],
       [0.04472824+0.j],
       [0.03860941+0.j],
       [0.04177482+0.j],
       [0.124896  +0.j],
       [0.03872253+0.j],
       [0.04460665+0.j],
       [0.04472824+0.j],
       [0.03872253+0.j],
       [0.04165755+0.j],
       [0.12489168+0.j],
       [0.03860941+0.j],
       [0.04460665+0.j],
       [0.12489168+0.j],
       [0.04189225+0.j]])

In [33]:
np.sum(P_init_GHZ)

(0.9999999999999998+0j)

In [34]:
np.sum(prob_operator(2, "Tetra", 4, gate, 1) @ P_init)

NameError: name 'P_init' is not defined

In [ ]:
%store AG
np.pi

Code could be faster if we actually started with all-complex numbers (also probabilities) in the first place and therefore without type-errors (i.e. complex+real). This works reasonably well now. Check for bigger operators

# Now look at the 2-qubit gates:

First encode the gates in the conventional way in the vector formalism:

In [35]:
CNOT = np.array([[0, 1, 0, 0],
                 [1, 0, 0, 0],
                 [0, 0, 1, 0],
                 [0, 0, 0, 1]])

SWAP = np.array([[1, 0, 0, 0],
                 [0, 0, 1, 0], 
                 [0, 1, 0, 0], 
                 [0, 0, 0, 1]])

phase = np.pi

CPHASE = np.array([[1, 0, 0, 0],
                   [0, 1, 0, 0],
                   [0, 0, 1, 0],
                   [0, 0, 0, np.exp(1.0j*phase)]])

Translate the gates to the POVM formalism:

In [36]:
def prob_operator_2qubits(n_qubits, POVM_type, n_outcomes, gate):
    
    n_mb_outcomes = n_outcomes**n_qubits
    measurements = list(product([1, 2, 3, 4], repeat=n_qubits))

    
    if POVM_type == "Tetra":
        POVM = POVM_tetra
        
    elif POVM_type == "Tetra_rot":
        POVM = POVM_tetra_rot

    overlap_mat = overlap(n_mb_outcomes, measurements, POVM_type)
    overlap_inv = np.linalg.inv(overlap_mat)
        
    mb_gate = gate
    
    O_mat = np.zeros((n_mb_outcomes, n_mb_outcomes), dtype=np.complex_)
    
    for i in range(n_mb_outcomes):
        for j in range(n_mb_outcomes):
            for k in range(n_mb_outcomes):
                O_mat[i][j] += np.trace(mb_gate @ many_body_POVM(ID_to_outcome(k, measurements), POVM) 
                                        @ dagger(mb_gate) @ many_body_POVM(ID_to_outcome(i, measurements), POVM)) * overlap_inv[k][j] 
    
    return(O_mat)

In [37]:
P_op_CNOT = prob_operator_2qubits(2, "Tetra_rot", 4, CNOT)
P_op_CPHASE =  prob_operator_2qubits(2, "Tetra_rot", 4, CPHASE)

In [38]:
%%capture
P_op_CNOT

Define a symmetric superposition state for 2 qubits:

In [39]:
two_q_sym = 1/2 * np.array([[1, 1, 1, 1]]).T

rho_init_two_q_sym = init_density_mat([(1, [two_q_sym])], 2)                            

P_init_two_q_sym =  rho_to_POVM_distr(rho_init_two_q_sym, 2, "Tetra_rot", 4)

Check if we have correctly implemented state + gate: 

1. determine density matrix rho_1 of state two_q_sym after the CPHASE gate
2. apply this gate in  the POVM formalism translate the resulting distribution back to density matrix rho_2
3. Check if rho_1 == rho_2



In [40]:
rho1_prime_two_q_sym = CPHASE @ rho_init_two_q_sym @ dagger(CPHASE)

In [41]:
P_prime_two_q_sym = P_op_CPHASE @ P_init_two_q_sym
rho2_prime_two_q_sym = POVM_distr_to_rho(P_prime_two_q_sym, 2, "Tetra_rot", 4)

print(rho2_prime_two_q_sym - rho1_prime_two_q_sym)

Stored 'measurements' (list)
[[ 3.88578059e-16-6.40803699e-19j -1.11022302e-16+2.77555756e-17j
   1.11022302e-16-8.32667268e-17j -1.66533454e-16-5.26505569e-17j]
 [-1.11022302e-16-2.77555756e-17j  2.22044605e-16-1.73736832e-17j
   1.11022302e-16-8.48511729e-17j  0.00000000e+00+3.06161700e-17j]
 [ 2.22044605e-16+9.71445147e-17j  1.11022302e-16+2.61711296e-17j
   0.00000000e+00-1.97390422e-17j -5.55111512e-17-1.35917284e-16j]
 [-1.11022302e-16+9.42839203e-17j  0.00000000e+00-2.86059436e-18j
  -5.55111512e-17+8.04061325e-17j  5.55111512e-17+4.65616945e-17j]]


We see that also for two-qubit gates everything works! 

In [42]:
%store P_init_two_q_sym
%store P_prime_two_q_sym
%store P_op_CPHASE



Stored 'P_init_two_q_sym' (ndarray)
Stored 'P_prime_two_q_sym' (ndarray)
Stored 'P_op_CPHASE' (ndarray)


### Now work in the machine learning notebook to check the 2-qubit gate update rule.

In [99]:
P_op_CPHASE

array([[ 9.99999848e-01+6.53054994e-19j,  1.40334203e-07-1.02766738e-18j,
         8.11934799e-06+3.91884103e-19j, -8.41139778e-06-6.70533194e-19j,
         1.40334203e-07-1.62656791e-18j,  4.49555681e-04+2.55757735e-19j,
        -2.40283496e-04-1.99649652e-19j, -2.09120530e-04+1.59246334e-18j,
         8.11934799e-06+3.32696972e-19j, -2.40283496e-04-7.08489878e-19j,
        -2.09125973e-04-2.47397236e-19j,  4.49555741e-04+5.96223863e-19j,
        -8.41139778e-06-4.46838531e-21j, -2.09120530e-04+9.53294268e-19j,
         4.49555741e-04+4.40888336e-19j, -2.40277787e-04-7.52064814e-19j],
       [ 4.49612106e-04-3.64620317e-18j,  9.99250721e-01-5.62617580e-18j,
        -2.86024135e-04-2.69957270e-17j, -3.13555839e-04+4.62253356e-17j,
        -8.82045545e-04-3.82255531e-18j,  7.14845981e-04-3.22449037e-18j,
         1.27429371e-02+3.89405606e-17j, -1.17109835e-02-2.42100675e-17j,
        -1.20105954e-02+3.70025678e-18j,  2.43348317e-02+6.28504240e-18j,
        -5.93243344e-04-2.56733215e-1